In [1]:
from MakeBoard import read_file
board_vals = read_file("d-10-06.txt.txt")
print(board_vals)

[[1, 0, 0, 0, 5, 0, 2, 6, 4, 0], [9, 0, 1, 2, 0, 0, 0, 0, 0, 7], [0, 0, 7, 5, 9, 1, 0, 0, 0, 0], [0, 0, 0, 0, 7, 4, 0, 1, 5, 0], [0, 2, 9, 0, 0, 0, 0, 3, 0, 5], [0, 4, 6, 3, 0, 0, 0, 0, 8, 0], [0, 0, 4, 7, 6, 0, 3, 0, 0, 0], [5, 1, 0, 4, 0, 0, 0, 0, 0, 8], [0, 6, 0, 0, 0, 7, 8, 5, 0, 4], [7, 9, 0, 0, 0, 8, 1, 0, 3, 0]]


In [2]:
class Cell:
        
    def __init__(self,board,row=0,col=0):
        self.row = row
        self.col = col
        self.board = board
        
    def __eq__(self, other):
        
        if isinstance(other,Cell) and other != None:
            return self.row == other.row and self.col == other.col
        else:
            return False
    
    def __hash__(self):
        return self.row*10+self.col
    
    def next_cell(self):
        row = self.row
        col = self.col
        
        try:
            while self.board[row][col] != 0:
                col += 1
                if col > len(self.board)-1:
                    col = 0
                    row += 1
        except IndexError:
            pass
        return Cell(self.board,row,col)

In [3]:
class Pair:
    
    def __init__(self,c1,c2):
        self.c1 = c1
        self.c2 = c2
        
    def __eq__(self, other):
        
        if other == None:
            return False
        if not isinstance(other,Cell):
            return False
        
        return other.left() == self.c1 and other.right() == self.c2
        
    def right(self):
        return self.c2
    def left(self):
        return self.c1

In [4]:
class DeleteQueue:
    
    def __init__(self,board):
        self.check_list = []
        self.delete_list = []
        self.board = board
        
    def add_delete(self,cell):
        self.add_row(cell)
        self.add_col(cell)
    
    def add_row(self,cell):
        for y in range(len(self.board)):
                if y != cell.col:
                    new_cell = Cell(self.board,cell.row,y)
                    self.check_list.append(new_cell)
                    
    def add_col(self,cell):
        for x in range(len(self.board)):
            if x != cell.row:
                new_cell = Cell(self.board,x,cell.col)
                self.check_list.append(new_cell)
    
    def update_all_domain(self,value,map_):
        for cell in self.check_list:
            self.update_domain(cell,value,map_)
    
    def update_domain(self,cell,value,map_):
        domains = map_[cell]
        
        if value in domains:
            self.delete_list.append(cell)
            domains.remove(value)
            map_[cell] = domains
            
    def check_empty_domain(self,map_):
        
        for cell in self.delete_list:
            if len(map_[cell]) == 0:
                return True
        return False
    
    def restore_domain(self,value,map_):
        for cell in self.delete_list:
            domain = map_[cell]
            domain.append(value)
            map_[cell] = domain
        self.delete_list.clear()

In [5]:
from collections import defaultdict
import time

class Player:
    def __init__(self, board):
        self.cell_map = defaultdict(list)
        self.queue = []
        self.ac3 = False
        self.board = board
        self.num_nodes_ac3 = 0
        self.num_recur_ac3 = 0
        self.num_nodes_forward = 0
        self.num_recur_forward = 0
        self.num_nodes_backtrack = 0
        self.num_recur_backtrack = 0
        
    def set_board(self,board):
        self.board = board

    def fill_cell_map(self):
        for i in range(len(self.board)):
            for j in range(len(self.board)):
                if self.board[i][j] == 0:
                    new_cell = Cell(self.board,i, j)
                    elements = self.fill_domain()
                    self.cell_map[new_cell] = elements
                #                     print(self.cell_map)
                else:
                    new_cell = Cell(self.board,i, j)
                    vals = [self.board[i][j]]
                    self.cell_map[new_cell] = vals

    def fill_domain(self):
        elements = []
        for i in range(1,len(self.board)+1):
            elements.append(i)
        return elements

    def print_domains(self):
        for i in range(len(self.board)):
            for j in range(len(self.board)):
                print("Domain cell (" + str(i) + "," + str(j) + "):", end=" ")
                cell = Cell(self.board,i, j);
                domain = self.cell_map[cell]
                # print(self.cell_map)
                # print(domain)
                for z in range(len(domain)):
                    print(str(domain[z]) + " ", end=" ")
                print("\n")

    def fill_Q(self):
        for i in range(len(self.board)):
            for j in range(len(self.board)):
                new_cell = Cell(self.board,i, j)
                self.add_to_queue(new_cell, True)

    def exists_in_queue(self, pair):
        for p in self.queue:

            if p == pair:
                return True
        return False

    def add_to_queue(self, cell, bo):
        self.add_row(cell, bo)

        self.add_col(cell, bo)

    def add_row(self, cell, bo):
        for i in range(len(self.board)):
            
            if i != cell.col:
                new_cell = Cell(self.board,cell.row, i)
                if bo:
                    pair = Pair(cell, new_cell)
                else:
                    pair = Pair(new_cell, cell)
                if not self.exists_in_queue(pair):
                    self.queue.append(pair)

    def add_col(self, cell, bo):
        for i in range(len(self.board)):
            if i != cell.row:
                new_cell = Cell(self.board,i, cell.col)
                if bo:
                    pair = Pair(cell, new_cell)
                else:
                    pair = Pair(new_cell, cell)
                if not self.exists_in_queue(pair):
                    self.queue.append(pair)
                    
                    
    def end_of_grid(self,board):
        for x in range(len(self.board)):
            for y in range(len(self.board)):
                if board[x][y] == 0:
                    return False
        return True
    
    def backtracking(self,cell):
        self.num_recur_backtrack += 1
        
        if self.end_of_grid(self.board):
            return True
        
        valuescell = self.cell_map[cell]
        value = 0
        
        for index in range(len(valuescell)):
            value = valuescell[index]
            self.board[cell.row][cell.col] = value
            if self.is_valid(cell,value):
                self.num_nodes_backtrack += 1
                if self.backtracking(cell.next_cell()):
                    return True
        self.board[cell.row][cell.col] = 0
        return False
    
    def is_valid(self,cell,value):
        
        for y in range(len(self.board)):
            if y != cell.col:
                if self.board[cell.row][y] == value:
                    return False
        
        for x in range(len(self.board)):
            if x != cell.row:
                if self.board[x][cell.col] == value:
                    return False
        return True
            
                    
    def forward_checking(self,cell):
        self.num_recur_forward += 1
        
        if self.end_of_grid(self.board):
            return True
        
        del_queue = DeleteQueue(self.board)
        del_queue.add_delete(cell)
        values_cell = self.cell_map[cell]
        
        value = 0
        
        for index in range(len(values_cell)):
            value = values_cell[index]
            del_queue.update_all_domain(value,self.cell_map)
            
            if del_queue.check_empty_domain(self.cell_map):
                del_queue.restore_domain(value,self.cell_map)
            
            else:
                new_domain = [value]
                self.cell_map[cell] = new_domain
                self.board[cell.row][cell.col] = value
                
                self.num_nodes_forward += 1
                
                if self.forward_checking(cell.next_cell()):
                    return True
                else:
                    del_queue.restore_domain(value,self.cell_map)
                    self.cell_map[cell] = values_cell
        self.board[cell.row][cell.col] = 0
        
        return False
    
    def print_board(self):
        for x in range(len(self.board)):
            for y in range(len(self.board)):
                print(self.board[x][y],end=" ")
            print("\n")
        

    def Ac3(self):
        self.ac3 = True
        solution_exists = True
        changed = True
        self.fill_cell_map()
        self.fill_Q()

        while len(self.queue) > 0 and solution_exists:
            self.num_recur_ac3 += 1
            pair = self.queue.pop(0)

            changed = False
            value_cell1 = self.cell_map[pair.left()]
            
            value_cell2 = self.cell_map[pair.right()]
            
            for i in reversed(range(len(value_cell1))):
                
                if self.delete_value(value_cell2, value_cell1[i]):
                    value_cell1.remove(value_cell1[i])
                    changed = True
                    
            if len(value_cell1) == 0:
                solution_exists = True
                
            if changed:
                self.cell_map[pair.left()] = value_cell1
                self.add_to_queue(pair.left(), False)
                self.num_nodes_ac3 += 1
                
        self.print_domains()
        if not solution_exists:
            print("There is no solution for this problem\n")
            return False
        return True

    def delete_value(self, values, value):
        for val in values:
            if val != value:
                return False
        return True

    def run_ac3(self):
        time1 = time.time()
        if self.Ac3():
            print("Solution Exists!!! :D")
        time2 = time.time()
        print("Number of Nodes {}".format(self.num_nodes_ac3))
        print("Number of Backtrack {}".format(self.num_recur_ac3))
        print("Time Elapsed " + str(time2 - time1))
        
    def run_forward_checking(self):
        if not self.ac3:
            self.fill_cell_map()
        cell = Cell(self.board)
        time1 = time.time()
        self.forward_checking(cell.next_cell())
        time2 = time.time()
        print("Number of Nodes {}".format(self.num_nodes_forward))
        print("Number of Backtrack {}".format(self.num_recur_forward))
        print("Time Taken for Forward Checking {}".format(time2-time1))
    def run_backtracking(self):
        if not self.ac3:
            self.fill_cell_map()
        cell = Cell(self.board)
        time1 = time.time()
        self.backtracking(cell.next_cell())
        time2 = time.time()
        print("Number of Nodes {}".format(self.num_nodes_backtrack))
        print("Number of Backtrack {}".format(self.num_recur_backtrack))
        print("Time Taken for Backtrack Checking {}".format(time2-time1))

In [6]:
obj = Player(board_vals.copy())

obj.run_ac3()

# obj.print_board()
# obj.run_backtracking()
# obj.print_board()

# obj.print_board()
# obj.run_forward_checking()
# obj.print_board()







Domain cell (0,0): 1  

Domain cell (0,1): 3  7  8  10  

Domain cell (0,2): 3  8  10  

Domain cell (0,3): 8  9  10  

Domain cell (0,4): 5  

Domain cell (0,5): 3  9  10  

Domain cell (0,6): 2  

Domain cell (0,7): 6  

Domain cell (0,8): 4  

Domain cell (0,9): 3  9  10  

Domain cell (1,0): 9  

Domain cell (1,1): 3  5  8  10  

Domain cell (1,2): 1  

Domain cell (1,3): 2  

Domain cell (1,4): 3  4  8  10  

Domain cell (1,5): 3  5  6  10  

Domain cell (1,6): 4  5  6  10  

Domain cell (1,7): 4  8  10  

Domain cell (1,8): 6  10  

Domain cell (1,9): 7  

Domain cell (2,0): 2  3  4  6  8  10  

Domain cell (2,1): 3  8  10  

Domain cell (2,2): 7  

Domain cell (2,3): 5  

Domain cell (2,4): 9  

Domain cell (2,5): 1  

Domain cell (2,6): 4  6  10  

Domain cell (2,7): 2  4  8  10  

Domain cell (2,8): 2  6  10  

Domain cell (2,9): 2  3  6  10  

Domain cell (3,0): 2  3  6  8  10  

Domain cell (3,1): 3  8  10  

Domain cell (3,2): 2  3  8  10  

Domain cell (3,3): 6  8  9  10  